We need the ics code for stations so that we can look them up on the TFL Journeyplanner

In [1]:
# Use SQLAlchemy to write each one to the database

from mylibrary.connections import Automapped_Base, conn, engine,session
from mylibrary.secrets import app_id, app_key
Stations = Automapped_Base.classes.all_stations

/Users/robinlinacre/anaconda/lib/python3.6/site-packages/sqlalchemy/dialects/postgresql/base.py:2526: SAWarning: Did not recognize type 'geometry' of column 'geom'
  (attype, name))


In [2]:
# Grab a list of all the ids of records which need to be matched
nlcs = session.query(Stations.nlc).filter(Stations.tfl_response != None).all()

In [15]:
import json
import pandas as pd
def process_icsCode(station):
   
    tfl_response_json = json.loads(station.tfl_response)
    
    return_object = {}
    
    if len(tfl_response_json["places"])==0:
        return_object["icscode"] = None
        return_object["icscode_status"] = "failed - no places in json"
        return return_object
    
    def contains_rail(x):
        return "national-rail" in x
    
    #Try to find a station of type "NaptanRailStation","NaptanRailAccessArea", sort by distance
    try:
        df = pd.DataFrame(tfl_response_json["places"]).sort_values("distance")
        df = df[df["stopType"].isin(["NaptanRailStation","NaptanRailAccessArea", "NaptanPublicBusCoachTram"])]
        df = df[df["placeType"] == "StopPoint"]
        df = df[df["modes"].apply(contains_rail)]
        df = df[pd.notnull(df["icsCode"])]
        return_object["icscode"] = df.iloc[0]["icsCode"]
        return_object["icscode_status"] = "ok"
        return return_object
    except:
        pass

    #Make a last ditch attempt if that didn't work - based on icscodes in a format that seem to represent valid stations.
    try:
        df = pd.DataFrame(tfl_response_json["places"]).sort_values("distance")
        f1 = df["icsCode"].astype(str).str[:2] == "10"
        f2 = df["icsCode"].astype(str).str[:2] == "90"
        df = df[f1|f2]

        return_object["icscode"] = df.iloc[0]["icsCode"]
        return_object["icscode_status"] = "ok"
        return return_object
    except:
        return_object["icscode"] = None
        return_object["icscode_status"] = "failed - during filtering"
        return return_object
    

    return_object["icscode"] = None
    return_object["icscode_status"] = "failed - after filtering, no match"
    return return_object

In [16]:
# Iterate through adding icscode
for nlc in nlcs:
    station = session.query(Stations).filter(Stations.nlc == nlc).one()
    ics_object = process_icsCode(station)
    station.icscode = ics_object["icscode"]
    station.icscode_status = ics_object["icscode_status"]
    session.add(station)
    session.commit()

,$type,additionalProperties,commonName,distance,icsCode,id,lat,lineGroup,lineModeGroups,lines,lon,modes,naptanId,placeType,stationNaptan,status,stopType
0,"Tfl.Api.Presentation.Entities.StopPoint, Tfl.A...",[],York Rail Station,108.165582,90008178,9100YORK,53.957966,[],[],[],-1.093182,[national-rail],9100YORK,StopPoint,910GYORK,True,NaptanRailAccessArea
1,"Tfl.Api.Presentation.Entities.StopPoint, Tfl.A...",[],York Rail Station,108.165582,90008178,9100YORK0,53.957966,[{'$type': 'Tfl.Api.Presentation.Entities.Line...,[{'$type': 'Tfl.Api.Presentation.Entities.Line...,[{'$type': 'Tfl.Api.Presentation.Entities.Iden...,-1.093182,[national-rail],9100YORK0,StopPoint,910GYORK,True,NaptanRailAccessArea
2,"Tfl.Api.Presentation.Entities.StopPoint, Tfl.A...",[],York Rail Station,108.165582,90008178,910GYORK,53.957966,[{'$type': 'Tfl.Api.Presentation.Entities.Line...,[{'$type': 'Tfl.Api.Presentation.Entities.Line...,[{'$type': 'Tfl.Api.Presentation.Entities.Iden...,-1.093182,[national-rail],910GYORK,StopPoint,910GYORK,True,NaptanRailStation


In [17]:
sql = """
select * from tt_h.all_stations
"""
pd.read_sql(sql, conn)

,nlc,tlc,station_name,region,local_authority,constituency,os_grid_easting,os_grid_northing,station_facility_owner,station_group,...,explanation_of_large_change,source_for_explanation_of_large_change,geom,lat,lng,icscode,icscode_status,tfl_request,tfl_response,tfl_message
0,375,ECP,Energlyn & Churchill Park,Wales - Cymru,Caerffili - Caerphilly,Caerphilly,314957,187866,Arriva Trains Wales,None,...,None,None,0101000020346C0000000000003439134100000000D0EE...,51.583211,-3.228795,90002592,ok,https://api.tfl.gov.uk/Place?lat=51.5832111676...,"{""$type"":""Tfl.Api.Presentation.Entities.Places...",ok
1,506,AMR,Amersham,South East,Chiltern,Chesham and Amersham,496379,198188,London Underground,None,...,None,None,0101000020346C000000000000EC4B1E41000000006031...,51.674204,-0.607596,1000006,ok,https://api.tfl.gov.uk/Place?lat=51.6742041137...,"{""$type"":""Tfl.Api.Presentation.Entities.Places...",ok
2,546,CLW,Chorleywood,East,Three Rivers,South West Hertfordshire,502597,196090,London Underground,None,...,London Stations Methodology Change,None,0101000020346C00000000000014AD1E4100000000D0EF...,51.654247,-0.518320,1000049,ok,https://api.tfl.gov.uk/Place?lat=51.6542469304...,"{""$type"":""Tfl.Api.Presentation.Entities.Places...",ok
3,577,ZFD,Farringdon,London,Islington (London),Islington South and Finsbury,531560,181840,London Underground,LONDON BR,...,London Stations Methodology Change,None,0101000020346C000000000000D0382041000000008032...,51.520165,-0.105205,1000080,ok,https://api.tfl.gov.uk/Place?lat=51.5201653993...,"{""$type"":""Tfl.Api.Presentation.Entities.Places...",ok
4,598,HOH,Harrow-On-The-Hill,London,Harrow (London),Harrow West,515314,188012,London Underground,None,...,London Stations Methodology Change,None,0101000020346C000000000000C8731F410000000060F3...,51.579184,-0.337225,1000102,ok,https://api.tfl.gov.uk/Place?lat=51.5791839298...,"{""$type"":""Tfl.Api.Presentation.Entities.Places...",ok
5,687,RIC,Rickmansworth,East,Three Rivers,South West Hertfordshire,505745,194597,London Underground,None,...,London Stations Methodology Change,None,0101000020346C00000000000044DE1E410000000028C1...,51.640245,-0.473283,1000193,ok,https://api.tfl.gov.uk/Place?lat=51.6402450878...,"{""$type"":""Tfl.Api.Presentation.Entities.Places...",ok
6,991,BKR,Blackridge,Scotland,West Lothian,Linlithgow and East Falkirk,290582,667064,ScotRail,None,...,None,None,0101000020346C00000000000058BC114100000000705B...,55.884250,-3.750797,90000666,ok,https://api.tfl.gov.uk/Place?lat=55.8842499236...,"{""$type"":""Tfl.Api.Presentation.Entities.Places...",ok
7,539,CFO,Chalfont & Latimer,South East,Chiltern,Chesham and Amersham,499647,197573,London Underground,None,...,None,None,0101000020346C000000000000FC7E1E4100000000281E...,51.668107,-0.560526,1000042,ok,https://api.tfl.gov.uk/Place?lat=51.6681067514...,"{""$type"":""Tfl.Api.Presentation.Entities.Places...",ok
8,990,ARM,Armadale,Scotland,West Lothian,Linlithgow and East Falkirk,294050,667140,ScotRail,None,...,None,None,0101000020346C00000000000088F2114100000000085C...,55.885708,-3.695413,90000237,ok,https://api.tfl.gov.uk/Place?lat=55.8857082879...,"{""$type"":""Tfl.Api.Presentation.Entities.Places...",ok
9,1003,BSC,Bescot Stadium,West Midlands,Sandwell,West Bromwich East,400700,296140,London Midland Trains,None,...,None,None,0101000020346C000000000000F0741841000000003013...,52.563091,-1.991105,90001076,ok,https://api.tfl.gov.uk/Place?lat=52.5630914887...,"{""$type"":""Tfl.Api.Presentation.Entities.Places...",ok
